## 0. Imports

In [1]:
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
import torch
import time
import json
from tqdm.notebook import tqdm

from dataset import LoadDataset
from MSGModel import Generator, Discriminator, DownSampleImage
from trainMSG import train_generator, train_discriminator, evaluate
from settings import DEVICE, MEAN, STD, BATCH_SIZE, LATENT_SIZE, EPOCHS, TRAINNING_DS_SIZE, RELOAD_DS_PER_EPOCH, LOSS_FN, OPTIMIZER, BETAS, D_LR, D_LR_DECAY, G_LR, G_LR_DECAY, CORES, LOAD_MODELS

# torch.set_num_threads(4)

%load_ext autoreload
%autoreload 2

## 1. Dataset

In [2]:
ds = LoadDataset('../data/train', MEAN, STD, length=TRAINNING_DS_SIZE)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)

val_ds = LoadDataset('../data/val', MEAN, STD)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)

print(f"Dataset size: {len(ds)}, loader size: {len(loader)}")
print(f"Validation dataset size: {len(val_ds)}, validation loader size: {len(val_loader)}")

Dataset size: 92032, loader size: 11504
Validation dataset size: 192, validation loader size: 24


## 2. Models

In [3]:
G, D, downsampler = Generator(), Discriminator(), DownSampleImage()
# print(G)
# print(D)
print(f"Num parameters in Discriminator: {sum(p.numel() for p in D.parameters())}")
print(f"Num parameters in Generator: {sum(p.numel() for p in G.parameters())}")

Num parameters in Discriminator: 23137153
Num parameters in Generator: 23053781


## 3. Training

In [4]:
d_losses = []
g_losses = []
r_scores = []
f_scores = []
d_optimizer = OPTIMIZER(D.parameters(), D_LR, betas=BETAS)
g_optimizer = OPTIMIZER(G.parameters(), G_LR, betas=BETAS)

try:
    if LOAD_MODELS:
        D.load_state_dict(torch.load('models/discriminator.pth'))
        G.load_state_dict(torch.load('models/generator.pth'))
        print("Model loaded")
    else:
        print("Trainning new model")
except:
    LOAD_MODEL = False
    print("Trainning new model")
    pass

D.to(DEVICE)
G.to(DEVICE)

if not LOAD_MODELS:
    with open("log.json", "w") as fh:
        json.dump({"epochs": 0}, fh)
    start_epochs = 0
else:
    with open("log.json", "r") as fh:
        log = json.load(fh)
    start_epochs = log['epochs']
    for e in range(1, start_epochs+1):
        d_losses.append(log[str(e)]['d_loss'])
        g_losses.append(log[str(e)]['g_loss'])
        r_scores.append(log[str(e)]['r_score'])
        f_scores.append(log[str(e)]['f_score'])

    d_optimizer.param_groups[0]['lr'] -= D_LR_DECAY * start_epochs
    g_optimizer.param_groups[0]['lr'] -= G_LR_DECAY * start_epochs
    
    latent = torch.randn(BATCH_SIZE, *LATENT_SIZE).to(DEVICE)
    fake_images = G(latent)[0] * 0.5 + 0.5
    save_image(fake_images, 'current.png')

print(f"Start at epochs {start_epochs}")

Model loaded
Start at epochs 39


In [5]:
for epochs in range(start_epochs, EPOCHS):
    start = time.time()
    # Reload dataset
    if RELOAD_DS_PER_EPOCH:
        ds = LoadDataset('../data/train', MEAN, STD, length=TRAINNING_DS_SIZE)
        loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)
    

    for batch_images in tqdm(loader):
        batch_images = batch_images.to(DEVICE)
        d_loss = train_discriminator(G, D, downsampler, batch_images, LOSS_FN, d_optimizer, LATENT_SIZE, BATCH_SIZE, DEVICE)
        g_loss = train_generator(G, D, LOSS_FN, g_optimizer, LATENT_SIZE, BATCH_SIZE, DEVICE)
        
    d_loss, g_loss, r_score, f_score, fake_images = evaluate(G, D, downsampler, LOSS_FN, val_loader, LATENT_SIZE, BATCH_SIZE, DEVICE)
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    r_scores.append(r_score)
    f_scores.append(f_score)
    
    torch.save(G.state_dict(), 'models/generator.pth')
    torch.save(D.state_dict(), 'models/discriminator.pth')
    
    d_loss = round(d_loss,5)
    g_loss = round(g_loss,5)
    r_score = round(r_score,5)
    f_score = round(f_score,5)

    d_optimizer.param_groups[0]['lr'] -= D_LR_DECAY
    g_optimizer.param_groups[0]['lr'] -= G_LR_DECAY

    elapsed = round(time.time()-start, 2)
    print(f"Epoch #{epochs+1}: d_loss={d_loss}, g_loss={g_loss}, r_score={r_score}, f_score={f_score} elapsed={elapsed} s")
    
    with open("log.json", "r") as fh:
        log = json.load(fh)
    log["epochs"] += 1
    log[epochs+1] = {"d_loss": d_loss, "g_loss": g_loss, "r_score": r_score, "f_score": f_score, "elapsed": elapsed}
    
    with open("log.json", "w") as fh:
        json.dump(log, fh)
    
    save_image(fake_images, f'output/{epochs+1}.png')

  0%|          | 0/11504 [00:00<?, ?it/s]

Epoch #40: d_loss=0.71647, g_loss=2.69979, r_score=0.67862, f_score=0.12022 elapsed=4998.07 s


  0%|          | 0/11504 [00:00<?, ?it/s]

Epoch #41: d_loss=0.13717, g_loss=8.22196, r_score=0.87877, f_score=0.00144 elapsed=4985.12 s


  0%|          | 0/11504 [00:00<?, ?it/s]

...

  0%|          | 0/11504 [00:00<?, ?it/s]

Epoch #73: d_loss=0.30318, g_loss=10.8112, r_score=0.81948, f_score=7e-05 elapsed=4968.3 s


  0%|          | 0/11504 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001DBADFA0E50>
Traceback (most recent call last):
  File "C:\Users\Jerry\miniconda3\envs\jupyter\lib\site-packages\torch\utils\data\dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "C:\Users\Jerry\miniconda3\envs\jupyter\lib\site-packages\torch\utils\data\dataloader.py", line 1474, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\Jerry\miniconda3\envs\jupyter\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "C:\Users\Jerry\miniconda3\envs\jupyter\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
latent = 